In [1]:
import numpy as np
from gurobipy import *
from scipy.stats import beta
from scipy.stats import norm
#import matplotlib.pyplot as plt
import random
import time

def RP(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit):
    

    num_i = M
    num_j = N
    num_t = T

    bar_I = np.array([1 for i in range(num_i)])
    # I_binary,initial solution for benders

    Z = np.array([(300*num_j)/num_i for i in range(num_i)])
    # Z'i_capacity
    
    if demandtype=='constant':
        demand = 200+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_t*num_j, 0.01), 2, 4)
        zeta=demand.reshape(num_t,num_j)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(zeta[t,j],0)
    if demandtype=='seasonal':
        demand=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        zeta=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        for t in range(num_t):
            for j in range(num_j):
                if (t-1)%4==0:
                    demand[t] = 400+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
                else:
                    demand[t] = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(demand[t,j],2)

    zetau=zeta
    #U'tj 需求均值

    zetaup=1.1*zetau
    #zetau'jt 需求上界

    zetalow=0.9*zetau
    #zetal'jt 需求下界

    eta = price
    # eta_price

    c =procost
    # ci_productioncost

    C =invcost
    # Cj_inventory cost

    d= transcost
    #dij_transportationcost
    
    F= np.array([20 for i in range(num_i)])
    #Fi_fixed construction cost

    f=desiredfillrate
    #ftj_target of fill rate

    R=profit
    #target profit

    num_k = 3
    a = [0.3909, 0.01298,0]
    b = [0, 0.8701, 1]
    #piecewise linear function,k=3

    epsilon=0.01


    params = dict()
    params["num_i"] = num_i
    params["num_j"] = num_j
    params["num_t"] = num_t
    params["Z"] = Z
    #params["P"] = P
    params["zetau"] = zetau
    params["zetaup"] = zetaup
    params["zetalow"] = zetalow
    params["eta"] = eta
    params["c"] = c
    params["C"] = C
    params["d"] = d
    params["F"] = F
    params["f"] = f
    params["R"] = R
    params["a"] = a
    params["b"] = b
    params["epsilon"] = epsilon
    params["bar_I"] = bar_I

    mip = Model()
    #决策变量
    y = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)], 
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="y")
    P = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)],
                vtype=GRB.CONTINUOUS, name="P")
    I = mip.addVars([i for i in range(num_i)], 
                vtype=GRB.BINARY, name="I")

    l = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="l") 

    D = mip.addVars([j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="D")

    A = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="A") 
    B = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="B")
    g = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="g") 
    h = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="h") 
    o = mip.addVars([t for t in range(num_t)], 
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="o")
    s = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="s")

    #目标函数
    mip.setObjective(
    (quicksum(D[j]
        for j in range(num_j))*eta
     -quicksum((d[i][j])*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
     -quicksum(c[i]*P[t,i]
        for t in range(num_t)
        for i in range(num_i))
     -quicksum(F[i]*I[i] 
        for i in range(num_i))
     -quicksum(C[j]*l[t,j]
        for t in range(num_t)
        for j in range(num_j))
    ),GRB.MAXIMIZE
    )

    #约束
    c0a=mip.addConstrs(
    (quicksum(
        -o[t,j]*zetaup[t,j]
        +s[t,j]*zetalow[t,j]
        for t in range(num_t))
     -D[j]
    )>=0
    for j in range(num_j)
    )
        
    c0b=mip.addConstrs(
    -o[t,j]+s[t,j]==1
    for j in range(num_j)
    for t in range(num_t)
    )

    c1 = mip.addConstrs(
    (D[j]
    -quicksum(y[t,i,j]
             for t in range(num_t)
             for i in range(num_i))
    )<=0
    for j in range(num_j)
    )
    #所有涉及n,t的地方都要+1         
                 
    c3a=mip.addConstrs(
    quicksum(
        -g[t,n,j]*zetaup[n,j]+h[t,n,j]*zetalow[n,j]
        for n in range(t+1))
    +l[t,j]
    -quicksum(y[n,i,j]
             for n in range(t+1)
             for i in range(num_i))
    >= 0
    for j in range(num_j)
    for t in range(num_t)
    )
    #所有涉及n的地方都要+1

    c3b=mip.addConstrs(
    -g[t,n,j]+h[t,n,j] == 1
    for j in range(num_j)
    for t in range(num_t)
    for n in range(t+1)
    )    

    c4 = mip.addConstrs( 
    quicksum(
        y[t,i,j] for j in range(num_j)
    ) <= P[t,i]
    for i in range(num_i)
    for t in range(num_t)
    )

    c5 = mip.addConstrs( 
    P[t,i]-Z[i]*I[i]<=0
    for i in range(num_i)
    for t in range(num_t)
    )

    mip.optimize()

    # Print solution
    #print("Optimal solution of decision variables")
    #for v in mip.getVars():
    #    print (v.varName, v.x)


    obj = mip.getObjective()
    print("Objective value")
    print(obj.getValue())
    return obj
    
    fullfill=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fmean=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fup=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    flow=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
        #fullfill[t,j]=sum(y[t,i,j] for i in range(num_i))
            fullfill[t,j]=max(sum(
            (sum(y[n,i,j].X for i in range(num_i))-zetau[n,j])
            for n in range(t)
                )+sum(y[t,i,j].X for i in range(num_i)),0)
            fmean[t,j]=round(fullfill[t,j]/zetau[t,j],2)
            fup[t,j]=round(fullfill[t,j]/zetaup[t,j],2)
            flow[t,j]=round(fullfill[t,j]/zetalow[t,j],2)
    print("fullfilled demand")
    print(fullfill)
    print("actual demand")
    print(zetau)
    print("fill rate compared to mean of demand")
    print(fmean)
    print("fill rate compared to upper bound of demand")
    print(fup)
    print("fill rate compared to lower bound of demand")
    print(flow)
    ###################################
    L=np.array([[0 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            L[t,j]=max(sum(
            (sum(
            y[n,i,j].x for i in range(num_i)
            )-zetau[n,j]) 
            for n in range(t+1)
        ),0)
    print("Inventory level of each store")
    print(L)  
    ###################################
    profit=(
     sum(D[j].x
        for j in range(num_j))*eta
     -sum((d[i][j])*y[t,i,j].x
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
     -sum(c[i]*P[t,i].x
        for t in range(num_t)
        for i in range(num_i))
     -sum(F[i]*I[i].x
        for i in range(num_i))
     -sum(C[j]*l[t,j].x
        for t in range(num_t)
        for j in range(num_j))
    )
    print("profit")
    print(profit)
    #################################
    num1=0
    num2=0
    num3=0
    for t in range(num_t):
        for j in range(num_j):
            if fmean[t,j]<f[t,j]:
                num1+=1
            if fup[t,j]<f[t,j]:
                num2+=1
            if flow[t,j]<f[t,j]:
                num3+=1
    per1=num1/(num_t*num_j)
    per2=num2/(num_t*num_j)
    per3=num3/(num_t*num_j)
    print("violation probability")
    print(round(per1*100,2))
    print(round(per2*100,2)) 
    print(round(per3*100,2))
    ################################
    LL=L.reshape(num_t*num_j,1).tolist()
    print('upper inventory level')
    print(max(LL))
    print('average inventory level')
    print(np.mean(LL))
    ##################################
    vio=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    vioup=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    violow=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            vio[t,j]=max(f[t,j]-fmean[t,j],0.000)
            vioup[t,j]=max(f[t,j]-fup[t,j],0.000)
            violow[t,j]=max(f[t,j]-flow[t,j],0.000)
    #没达到的程度        
    viomean=np.mean(vio)
    viostd=np.std(vio)
    vioVaR95 = norm.ppf(1-0.95, viomean, viostd)
    vioVaR99 = norm.ppf(1-0.99, viomean, viostd)
    vioupmean=np.mean(vioup)
    vioupstd=np.std(vioup)
    vioupVaR95 = norm.ppf(1-0.95, vioupmean, vioupstd)
    vioupVaR99 = norm.ppf(1-0.99, vioupmean, vioupstd)
    violowmean=np.mean(violow)
    violowstd=np.std(violow)
    violowVaR95 = norm.ppf(1-0.95, violowmean, violowstd)
    violowVaR99 = norm.ppf(1-0.99, violowmean, violowstd)

    print('mean/std/VaR95/VaR99 compared to mean')
    print(round(viomean,2))
    print(round(viostd,2))
    print(round(vioVaR95,2))
    print(round(vioVaR99,2))
    print('mean/std/VaR95/VaR99 compared to upper')
    print(round(vioupmean,2))
    print(round(vioupstd,2))
    print(round(vioupVaR95,2))
    print(round(vioupVaR99,2))
    print('mean/std/VaR95/VaR99 compared to lower')
    print(round(violowmean,2))
    print(round(violowstd,2))
    print(round(violowVaR95,2))
    print(round(violowVaR99,2))



In [2]:
import numpy as np
from gurobipy import *
from scipy.stats import beta
from scipy.stats import norm
import matplotlib.pyplot as plt
import random
import time

def RPF(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit):
    
    
    num_i = M
    num_j = N
    num_t = T

    bar_I = np.array([1 for i in range(num_i)])
    # I_binary,initial solution for benders

    Z = np.array([(400*num_j)/num_i for i in range(num_i)])
    # Z'i_capacity
    
    if demandtype=='constant':
        demand = 200+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_t*num_j, 0.01), 2, 4)
        zeta=demand.reshape(num_t,num_j)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(zeta[t,j],0)
    if demandtype=='seasonal':
        demand=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        zeta=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        for t in range(num_t):
            for j in range(num_j):
                if (t-1)%4==0:
                    demand[t] = 400+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
                else:
                    demand[t] = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(demand[t,j],2)

    zetau=zeta
    #U'tj 需求均值

    zetaup=1.1*zetau
    #zetau'jt 需求上界

    zetalow=0.9*zetau
    #zetal'jt 需求下界
   
    eta = price
    # eta_price

    c =procost
    # ci_productioncost

    C =invcost
    # Cj_inventory cost

    d= transcost
    #dij_transportationcost
    
    F= np.array([20 for i in range(num_i)])
    #Fi_fixed construction cost

    f=desiredfillrate
    #ftj_target of fill rate

    R=profit
    #target profit

    num_k = 3
    a = [0.3909, 0.01298,0]
    b = [0, 0.8701, 1]
    #piecewise linear function,k=3

    epsilon=0.01


    params = dict()
    params["num_i"] = num_i
    params["num_j"] = num_j
    params["num_t"] = num_t
    params["Z"] = Z
    #params["P"] = P
    params["zetau"] = zetau
    params["zetaup"] = zetaup
    params["zetalow"] = zetalow
    params["eta"] = eta
    params["c"] = c
    params["C"] = C
    params["d"] = d
    params["F"] = F
    params["f"] = f
    params["R"] = R
    params["a"] = a
    params["b"] = b
    params["epsilon"] = epsilon
    params["bar_I"] = bar_I

    mip = Model()
    #决策变量
    y = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)], 
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="y")
    P = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)],
                vtype=GRB.CONTINUOUS, name="P")
    I = mip.addVars([i for i in range(num_i)], 
                vtype=GRB.BINARY, name="I")
    l = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="l")
    D = mip.addVars([j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="D")

    r = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="r") 
    w = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="w") 
    A = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="A") 
    B = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="B") 
    g = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="g") 
    h = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="h") 
    o = mip.addVars([t for t in range(num_t)], 
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="o")
    s = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="s")

    #目标函数
    mip.setObjective(
    (quicksum(D[j]
        for j in range(num_j))*eta
     -quicksum((d[i][j])*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
     -quicksum(c[i]*P[t,i]
        for t in range(num_t)
        for i in range(num_i))
     -quicksum(F[i]*I[i] 
        for i in range(num_i))
     -quicksum(C[j]*l[t,j]
        for t in range(num_t)
        for j in range(num_j))
    ),GRB.MAXIMIZE
    )

    #约束
    c0a=mip.addConstrs(
    (quicksum(
        -o[t,j]*zetaup[t,j]
        +s[t,j]*zetalow[t,j]
        for t in range(num_t))
     -D[j]
    )>=0
    for j in range(num_j)
    )

    c0b=mip.addConstrs(
    -o[t,j]+s[t,j]==1
    for j in range(num_j)
    for t in range(num_t)
    )

    c1 = mip.addConstrs(
    (D[j]
    -quicksum(y[t,i,j]
             for t in range(num_t)
             for i in range(num_i))
    )<=0
    for j in range(num_j)
    )
    #所有涉及n,t的地方都要+1         


    c2 = mip.addConstrs(
    (quicksum(
        r[t,n,j]*zetaup[n,j]-w[t,n,j]*zetalow[n,j]
        for n in range(t+1))
    -quicksum(y[n,i,j] 
        for i in range(num_i)
        for n in range(t+1))
    )<=0
    for j in range(num_j)
    for t in range(num_t)
    )
    #所有涉及n,t的地方都要+1         
    for j in range(num_j):
        for t in range(num_t):
            for n in range(t+1):                   
                if n==t:
                     mip.addConstr(
                     (r[t,n,j]-w[t,n,j]-f[t][j])==0
                     )
                if n<t:
                    mip.addConstr(
                    r[t,n,j]-w[t,n,j] == 1
                    ) 
                    
            
    c3a=mip.addConstrs(
        quicksum(
            -g[t,n,j]*zetaup[n,j]+h[t,n,j]*zetalow[n,j]
            for n in range(t+1))
        +l[t,j]
        -quicksum(y[n,i,j]
                 for n in range(t+1)
                 for i in range(num_i))
        >= 0
        for j in range(num_j)
        for t in range(num_t)
    )
    #所有涉及n的地方都要+1

    c3b=mip.addConstrs(
    -g[t,n,j]+h[t,n,j] == 1
    for j in range(num_j)
    for t in range(num_t)
    for n in range(t+1)
    )     

    c4 = mip.addConstrs( 
    quicksum(
        y[t,i,j] for j in range(num_j)
    ) <= P[t,i]
    for i in range(num_i)
    for t in range(num_t)
    )

    c5 = mip.addConstrs( 
    P[t,i]-Z[i]*I[i]<=0
    for i in range(num_i)
    for t in range(num_t)
    )


    mip.optimize()

    # Print solution
    #print("Optimal solution of decision variables")
    #for v in mip.getVars():
    #    print (v.varName, v.x)

    obj = mip.getObjective()
    print("Objective value")
    print(obj.getValue())
    return obj

    fullfill=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fmean=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fup=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    flow=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
        #fullfill[t,j]=sum(y[t,i,j] for i in range(num_i))
            fullfill[t,j]=max(sum(
            (sum(y[n,i,j].X for i in range(num_i))-zetau[n,j])
            for n in range(t)
                )+sum(y[t,i,j].X for i in range(num_i)),0)
            fmean[t,j]=round(fullfill[t,j]/zetau[t,j],2)
            fup[t,j]=round(fullfill[t,j]/zetaup[t,j],2)
            flow[t,j]=round(fullfill[t,j]/zetalow[t,j],2)
    print("fullfilled demand")
    print(fullfill)
    print("actual demand")
    print(zetau)
    print("fill rate compared to mean of demand")
    print(fmean)
    print("fill rate compared to upper bound of demand")
    print(fup)
    print("fill rate compared to lower bound of demand")
    print(flow)
    ###################################
    L=np.array([[0 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            L[t,j]=max(sum(
            (sum(
            y[n,i,j].x for i in range(num_i)
            )-zetau[n,j]) 
            for n in range(t+1)
        ),0)
    print("Inventory level of each store")
    print(L)  
    ###################################
    profit=(
     sum(D[j].x
        for j in range(num_j))*eta
     -sum((d[i][j])*y[t,i,j].x
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
     -sum(c[i]*P[t,i].x
        for t in range(num_t)
        for i in range(num_i))
     -sum(F[i]*I[i].x
        for i in range(num_i))
     -sum(C[j]*l[t,j].x
        for t in range(num_t)
        for j in range(num_j))
    )
    print("profit")
    print(profit)
    #################################
    num1=0
    num2=0
    num3=0
    for t in range(num_t):
        for j in range(num_j):
            if fmean[t,j]<f[t,j]:
                num1+=1
            if fup[t,j]<f[t,j]:
                num2+=1
            if flow[t,j]<f[t,j]:
                num3+=1
    per1=num1/(num_t*num_j)
    per2=num2/(num_t*num_j)
    per3=num3/(num_t*num_j)
    print("violation probability")
    print(round(per1*100,2))
    print(round(per2*100,2)) 
    print(round(per3*100,2))
    ################################
    LL=L.reshape(num_t*num_j,1).tolist()
    print('upper inventory level')
    print(max(LL))
    print('average inventory level')
    print(np.mean(LL))
    ##################################
    vio=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    vioup=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    violow=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            vio[t,j]=max(f[t,j]-fmean[t,j],0.000)
            vioup[t,j]=max(f[t,j]-fup[t,j],0.000)
            violow[t,j]=max(f[t,j]-flow[t,j],0.000)
    #没达到的程度        
    viomean=np.mean(vio)
    viostd=np.std(vio)
    vioVaR95 = norm.ppf(1-0.95, viomean, viostd)
    vioVaR99 = norm.ppf(1-0.99, viomean, viostd)
    vioupmean=np.mean(vioup)
    vioupstd=np.std(vioup)
    vioupVaR95 = norm.ppf(1-0.95, vioupmean, vioupstd)
    vioupVaR99 = norm.ppf(1-0.99, vioupmean, vioupstd)
    violowmean=np.mean(violow)
    violowstd=np.std(violow)
    violowVaR95 = norm.ppf(1-0.95, violowmean, violowstd)
    violowVaR99 = norm.ppf(1-0.99, violowmean, violowstd)

    print('mean/std/VaR95/VaR99 compared to mean')
    print(round(viomean,2))
    print(round(viostd,2))
    print(round(vioVaR95,2))
    print(round(vioVaR99,2))
    print('mean/std/VaR95/VaR99 compared to upper')
    print(round(vioupmean,2))
    print(round(vioupstd,2))
    print(round(vioupVaR95,2))
    print(round(vioupVaR99,2))
    print('mean/std/VaR95/VaR99 compared to lower')
    print(round(violowmean,2))
    print(round(violowstd,2))
    print(round(violowVaR95,2))
    print(round(violowVaR99,2))


In [3]:
import numpy as np
from gurobipy import *
from scipy.stats import beta
from scipy.stats import norm
import matplotlib.pyplot as plt
import random
import time

def RF(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit):
    
    num_i = M
    num_j = N
    num_t = T

    bar_I = np.array([1 for i in range(num_i)])
    # I_binary,initial solution for benders

    Z = np.array([(400*num_j)/num_i for i in range(num_i)])
    # Z'i_capacity
    
    if demandtype=='constant':
        demand = 200+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_t*num_j, 0.01), 2, 4)
        zeta=demand.reshape(num_t,num_j)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(zeta[t,j],0)
    if demandtype=='seasonal':
        demand=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        zeta=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        for t in range(num_t):
            for j in range(num_j):
                if (t-1)%4==0:
                    demand[t] = 400+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
                else:
                    demand[t] = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(demand[t,j],2)

    zetau=zeta
    #U'tj 需求均值

    zetaup=1.1*zetau
    #zetau'jt 需求上界

    zetalow=0.9*zetau
    #zetal'jt 需求下界

    eta = price
    # eta_price

    c =procost
    # ci_productioncost

    C =invcost
    # Cj_inventory cost

    d= transcost
    #dij_transportationcost
    
    F= np.array([20 for i in range(num_i)])
    #Fi_fixed construction cost

    f=desiredfillrate
    #ftj_target of fill rate

    R=profit
    #target profit

    num_k = 3
    a = [0.3909, 0.01298,0]
    b = [0, 0.8701, 1]
    #piecewise linear function,k=3

    epsilon=0.01


    params = dict()
    params["num_i"] = num_i
    params["num_j"] = num_j
    params["num_t"] = num_t
    params["Z"] = Z
    #params["P"] = P
    params["zetau"] = zetau
    params["zetaup"] = zetaup
    params["zetalow"] = zetalow
    params["eta"] = eta
    params["c"] = c
    params["C"] = C
    params["d"] = d
    params["F"] = F
    params["f"] = f
    params["R"] = R
    params["a"] = a
    params["b"] = b
    params["epsilon"] = epsilon
    params["bar_I"] = bar_I

    mip = Model()
    #决策变量
    y = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)], 
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="y")
    P = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)],
                vtype=GRB.CONTINUOUS, name="P")
    I = mip.addVars([i for i in range(num_i)], 
                vtype=GRB.BINARY, name="I")
    alpha=mip.addVars([t for t in range(num_t)],
                  [j for j in range(num_j)],
                  vtype=GRB.CONTINUOUS, name="alpha")

    S0= mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                lb=-1*GRB.INFINITY,
                vtype=GRB.CONTINUOUS, name="S0")
    S= mip.addVars([t for t in range(num_t)],
               [n for n in range(num_t)],
               [j for j in range(num_j)],
               lb=-1*GRB.INFINITY,
               vtype=GRB.CONTINUOUS, name="S")
    u= mip.addVars([k for k in range(num_k)],
               [t for t in range(num_t)],
               [n for n in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="u")
    v= mip.addVars([k for k in range(num_k)],
               [t for t in range(num_t)],
               [n for n in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="v")
    A = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="A") 
    B = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="B") 
    G = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="G") 
    H = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="H") 
    o = mip.addVars([t for t in range(num_t)], 
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="o")
    s = mip.addVars([t for t in range(num_t)],
                [n for n in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="s")
    pi = mip.addVar(vtype=GRB.CONTINUOUS, name="pi")

    #目标函数
    mip.setObjective(
    quicksum(
            alpha[t,j]
            for t in range(num_t)
            for j in range(num_j)
    ),GRB.MINIMIZE
    )

    #约束
    #第一组4个
    c1a=mip.addConstrs(
    (S0[t,j]+
    quicksum(zetau[n,j]*S[t,n,j] 
             for n in range(t+1))
    )>= 0
    for t in range(num_t)
    for j in range(num_j))

    c1b=mip.addConstrs(
    (quicksum(zetaup[n,j]*u[k,t,n,j]
             -zetalow[n,j]*v[k,t,n,j]
            for n in range(t+1))
    +S0[t,j]
    -a[k]*quicksum(y[n,i,j] 
                   for i in range(num_i)
                   for n in range(t+1))
    )<=0
    for k in range(num_k)
    for t in range(num_t)
    for j in range(num_j))

    #c1c,c1d
    for k in range(num_k):
        for t in range(num_t):
            for n in range(t+1):
                for j in range(num_j):
                    if n==(t):
                        mip.addConstr(
                        (u[k,t,n,j]-v[k,t,n,j]
                         -S[t,n,j]
                         -a[k]*f[t][j]
                         +b[k]*alpha[t,j]
                        )==0)
                    if n<(t):
                        mip.addConstr(
                        (u[k,t,n,j]-v[k,t,n,j]
                         -S[t,n,j]
                         -a[k]
                        )==0)

    #利润限制共四组
    cc=mip.addConstr(
    (quicksum(c[i]*P[t,i]
            for i in range(num_i)
            for t in range(num_t))
    +quicksum(F[i]*I[i]
             for i in range(num_i))
    -pi)==0    
    )

    #第二组3个
    c2a=mip.addConstr(
    (quicksum(
        -A[t,n,j]*zetaup[n,j]
        +B[t,n,j]*zetalow[n,j]
        for t in range(num_t)
        for j in range(num_j)
        for n in range(t+1))
    -R
    -quicksum(d[i][j]*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi
    -quicksum(C[j]*y[n,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        for n in range(t+1))
    )>=0
    )
    #c2b,c2c
    for t in range(num_t):
        for n in range(t+1):
            for j in range(num_j):
                if n==(t):
                    mip.addConstr(
                    (-A[t,n,j]+B[t,n,j]
                     -eta
                     -C[j]
                    )==0)
                if n<(t):
                    mip.addConstr(
                    (-A[t,n,j]+B[t,n,j]
                     -C[j]
                    )==0)

    #第三组 2个
    c3a=mip.addConstr(
    (quicksum(
        -G[t,j]*zetaup[t,j]+H[t,j]*zetalow[t,j]
        for t in range(num_t)
        for j in range(num_j))
    -R
    -quicksum(d[i][j]*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi)
    >=0
    )

    c3b=mip.addConstrs(
    -G[t,j]+H[t,j]-eta==0
    for j in range(num_j)
    for t in range(num_t)
    )

    #第四组 2个
    c4a=mip.addConstr(
    (quicksum(
        -o[t,n,j]*zetaup[n,j]
        +s[t,n,j]*zetalow[n,j]
        for t in range(num_t)
        for j in range(num_j)
        for n in range(t+1))
    -R
    +quicksum((eta-d[i][j])*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi
    -quicksum(C[j]*y[n,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        for n in range(t+1))
    )>=0
    )

    c4b=mip.addConstrs(
    -o[t,n,j]+s[t,n,j]-C[j]==0
    for j in range(num_j)
    for t in range(num_t)
    for n in range(t+1)
    )

    #第五组 1个
    c5=mip.addConstr(
    (quicksum((eta-d[i][j])*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi
    -R
    )>=0
    )
  
    #第六组
    c6 = mip.addConstrs( 
    (quicksum(
        y[t,i,j] for j in range(num_j)
    ) 
    -P[t,i]) <= 0
    for i in range(num_i)
    for t in range(num_t)
    )

    #第7个
    c7 = mip.addConstrs( 
    P[t,i]-Z[i]*I[i]<=0
    for i in range(num_i)
    for t in range(num_t)
    )

    #第八组
    c8 = mip.addConstrs( 
    y[t,i,j]>=0
    for i in range(num_i)
    for j in range(num_j)
    for t in range(num_t)
    )

    #number 9
    c9=mip.addConstrs(
    alpha[t,j] >= epsilon
    for t in range(num_t)
    for j in range(num_j))

    mip.optimize()

        # Print solution
    #print("Optimal solution of decision variables")
    #for v in mip.getVars():
    #    print (v.varName, v.x)

    obj = mip.getObjective()
    print("Objective value")
    print(obj.getValue())
    return obj

    fullfill=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fmean=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fup=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    flow=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
        #fullfill[t,j]=sum(y[t,i,j] for i in range(num_i))
            fullfill[t,j]=max(sum(
            (sum(y[n,i,j].X for i in range(num_i))-zetau[n,j])
            for n in range(t)
                )+sum(y[t,i,j].X for i in range(num_i)),0)
            fmean[t,j]=round(fullfill[t,j]/zetau[t,j],2)
            fup[t,j]=round(fullfill[t,j]/zetaup[t,j],2)
            flow[t,j]=round(fullfill[t,j]/zetalow[t,j],2)
    print("fullfilled demand")
    print(fullfill)
    print("actual demand")
    print(zetau)
    print("fill rate compared to mean of demand")
    print(fmean)
    print("fill rate compared to upper bound of demand")
    print(fup)
    print("fill rate compared to lower bound of demand")
    print(flow)
    ###################################
    L=np.array([[0 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            L[t,j]=max(sum(
            (sum(
            y[n,i,j].x for i in range(num_i)
            )-zetau[n,j]) 
            for n in range(t+1)
        ),0)
    print("Inventory level of each store")
    print(L)  
    ###################################
    profit1=(sum(eta*zetau[t,j] 
            for t in range(num_t)
            for j in range(num_j))
        -sum(d[i,j]*y[t,i,j].x
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j))
        -pi.x
        -sum(C[j]*y[n,i,j].x
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j)
            for n in range(t+1))
        +sum(C[j]*zetau[n,j]
            for t in range(num_t)
            for j in range(num_j)
            for n in range(t+1))
        )
    profit2=(sum(eta*zetau[t,j] 
            for t in range(num_t)
            for j in range(num_j))
        -sum(d[i,j]*y[t,i,j].x
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j))
        -pi.x
        )         
    profit3=(sum((eta-d[i,j])*y[t,i,j].x
             for t in range(num_t)
             for i in range(num_i)
             for j in range(num_j))
         -pi.x
         -sum(C[j]*y[n,i,j].x
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j)
            for n in range(t+1))
         +sum(C[j]*zetau[n,j]
            for t in range(num_t)
            for j in range(num_j)
            for n in range(t+1))
        )         
    profit4=(sum((eta-d[i,j])*y[t,i,j].x
             for t in range(num_t)
             for i in range(num_i)
             for j in range(num_j))
         -pi.x
    )
    print("profit1")
    print(profit1)
    print("profit2")
    print(profit2)
    print("profit3")
    print(profit3)
    print("profit4")
    print(profit4)

        #################################
    num1=0
    num2=0
    num3=0
    for t in range(num_t):
        for j in range(num_j):
            if fmean[t,j]<f[t,j]:
                num1+=1
            if fup[t,j]<f[t,j]:
                num2+=1
            if flow[t,j]<f[t,j]:
                num3+=1
    per1=num1/(num_t*num_j)
    per2=num2/(num_t*num_j)
    per3=num3/(num_t*num_j)
    print("violation probability")
    print(round(per1*100,2))
    print(round(per2*100,2)) 
    print(round(per3*100,2))
    ################################
    LL=L.reshape(num_t*num_j,1).tolist()
    print('upper inventory level')
    print(max(LL))
    print('average inventory level')
    print(np.mean(LL))
    ##################################
    vio=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    vioup=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    violow=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            vio[t,j]=max(f[t,j]-fmean[t,j],0.000)
            vioup[t,j]=max(f[t,j]-fup[t,j],0.000)
            violow[t,j]=max(f[t,j]-flow[t,j],0.000)
    #没达到的程度        
    viomean=np.mean(vio)
    viostd=np.std(vio)
    vioVaR95 = norm.ppf(1-0.95, viomean, viostd)
    vioVaR99 = norm.ppf(1-0.99, viomean, viostd)
    vioupmean=np.mean(vioup)
    vioupstd=np.std(vioup)
    vioupVaR95 = norm.ppf(1-0.95, vioupmean, vioupstd)
    vioupVaR99 = norm.ppf(1-0.99, vioupmean, vioupstd)
    violowmean=np.mean(violow)
    violowstd=np.std(violow)
    violowVaR95 = norm.ppf(1-0.95, violowmean, violowstd)
    violowVaR99 = norm.ppf(1-0.99, violowmean, violowstd)

    print('mean/std/VaR95/VaR99 compared to mean')
    print(round(viomean,2))
    print(round(viostd,2))
    print(round(vioVaR95,2))
    print(round(vioVaR99,2))
    print('mean/std/VaR95/VaR99 compared to upper')
    print(round(vioupmean,2))
    print(round(vioupstd,2))
    print(round(vioupVaR95,2))
    print(round(vioupVaR99,2))
    print('mean/std/VaR95/VaR99 compared to lower')
    print(round(violowmean,2))
    print(round(violowstd,2))
    print(round(violowVaR95,2))
    print(round(violowVaR99,2))


In [4]:
import numpy as np
from gurobipy import *
from scipy.stats import beta
from scipy.stats import norm
import matplotlib.pyplot as plt
import random
import time

def RFD(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit):
    
    
    num_i = M
    num_j = N
    num_t = T

    bar_I = np.array([1 for i in range(num_i)])
    # I_binary,initial solution for benders

    Z = np.array([(400*num_j)/num_i for i in range(num_i)])
    # Z'i_capacity
    
    if demandtype=='constant':
        demand = 200+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_t*num_j, 0.01), 2, 4)
        zeta=demand.reshape(num_t,num_j)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(zeta[t,j],0)
    if demandtype=='seasonal':
        demand=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        zeta=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        for t in range(num_t):
            for j in range(num_j):
                if (t-1)%4==0:
                    demand[t] = 400+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
                else:
                    demand[t] = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(demand[t,j],2)

    zetau=zeta
    #U'tj 需求均值

    zetaup=1.1*zetau
    #zetau'jt 需求上界

    zetalow=0.9*zetau
    #zetal'jt 需求下界

    eta = price
    # eta_price

    c =procost
    # ci_productioncost

    C =invcost
    # Cj_inventory cost

    d= transcost
    #dij_transportationcost
    
    F= np.array([20 for i in range(num_i)])
    #Fi_fixed construction cost

    f=desiredfillrate
    #ftj_target of fill rate

    R=profit
    #target profit

    num_k = 3
    a = [0.3909, 0.01298,0]
    b = [0, 0.8701, 1]
    #piecewise linear function,k=3

    epsilon=0.01


    params = dict()
    params["num_i"] = num_i
    params["num_j"] = num_j
    params["num_t"] = num_t
    params["Z"] = Z
    #params["P"] = P
    params["zetau"] = zetau
    params["zetaup"] = zetaup
    params["zetalow"] = zetalow
    params["eta"] = eta
    params["c"] = c
    params["C"] = C
    params["d"] = d
    params["F"] = F
    params["f"] = f
    params["R"] = R
    params["a"] = a
    params["b"] = b
    params["epsilon"] = epsilon
    params["bar_I"] = bar_I

    mip = Model()
    #决策变量
    q0 = mip.addVars([t for t in range(num_t)],
                 [i for i in range(num_i)], 
                 [j for j in range(num_j)],
                 lb=-1*GRB.INFINITY,
                 vtype=GRB.CONTINUOUS, name="q0")
    q = mip.addVars([t for t in range(num_t)],
                [tt for tt in range(num_t)],
                [i for i in range(num_i)], 
                [j for j in range(num_j)],
                lb=-1*GRB.INFINITY,
                vtype=GRB.CONTINUOUS, name="q")
    P = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)],
                vtype=GRB.CONTINUOUS, name="P")
    I = mip.addVars([i for i in range(num_i)],
                vtype=GRB.BINARY, name="I")
    alpha=mip.addVars([t for t in range(num_t)],
                  [j for j in range(num_j)],
                  vtype=GRB.CONTINUOUS, name="alpha")

    #####################################################
    S0= mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                lb=-1*GRB.INFINITY,
                vtype=GRB.CONTINUOUS, name="S0")
    S= mip.addVars([t for t in range(num_t)],
               [j for j in range(num_j)],
               lb=-1*GRB.INFINITY,
               vtype=GRB.CONTINUOUS, name="S")
    u= mip.addVars([k for k in range(num_k)],
               [t for t in range(num_t)],
               [n for n in range(num_t)],
               [nn for nn in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="u")
    v= mip.addVars([k for k in range(num_k)],
               [t for t in range(num_t)],
               [n for n in range(num_t)],
               [nn for nn in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="v")
    A= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [n for n in range(num_t)],
               [nn for nn in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="A")
    B= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [n for n in range(num_t)],
               [nn for nn in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="B")
    G= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="G")
    H= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="H")
    o= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [n for n in range(num_t)],
               [nn for nn in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="o")
    s= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [n for n in range(num_t)],
               [nn for nn in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="s")
    r= mip.addVars([tt for tt in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="r")
    w= mip.addVars([tt for tt in range(num_t)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="w")
    x= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [i for i in range(num_i)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="x")
    z= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [i for i in range(num_i)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="z")
    g= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [i for i in range(num_i)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="g")
    h= mip.addVars([t for t in range(num_t)],
               [tt for tt in range(num_t)],
               [i for i in range(num_i)],
               [j for j in range(num_j)],
               vtype=GRB.CONTINUOUS, name="h")
    pi = mip.addVar(vtype=GRB.CONTINUOUS, name="pi")

    #目标函数
    mip.setObjective(
    quicksum(
            alpha[t,j]
            for t in range(num_t)
            for j in range(num_j)
    ),GRB.MINIMIZE
    )

    #约束
    #第一组5个
    c1a=mip.addConstrs(
    (S0[t,j]+
    zetau[t,j]*S[t,j]
    )>= 0
    for t in range(num_t)
    for j in range(num_j))

    c1b=mip.addConstrs(
    (quicksum(zetaup[nn,j]*u[k,t,n,nn,j]
             -zetalow[nn,j]*v[k,t,n,nn,j]
            for n in range(t+1)
            for nn in range(t+1))
    +S0[t,j]
    -a[k]*quicksum(q0[n,i,j] 
                   for i in range(num_i)
                   for n in range(t+1))
    )<=0
    for k in range(num_k)
    for t in range(num_t)
    for j in range(num_j))

    #c1c,c1d,c1e
    for k in range(num_k):
        for t in range(num_t):
            for n in range(t+1):
                for nn in range(t+1):
                    for j in range(num_j):
                        if nn!=n:
                            mip.addConstr(
                            (u[k,t,n,nn,j]-v[k,t,n,nn,j]
                             +a[k]*quicksum(q[n,nn,i,j]
                                           for i in range(num_i))
                            )==0)
                        if nn==n:
                            if n<t:
                                mip.addConstr(
                                (u[k,t,n,nn,j]-v[k,t,n,nn,j]
                                 +a[k]*quicksum(q[n,nn,i,j]
                                           for i in range(num_i))
                                 -a[k]
                            )==0)
                            if n==t&nn==t:
                                mip.addConstr(
                                (u[k,t,n,nn,j]-v[k,t,n,nn,j]
                                 +a[k]*quicksum(q[n,nn,i,j]
                                           for i in range(num_i))
                                 -S[t,j]
                                 -a[k]*f[t,j]
                                 +b[k]*alpha[t,j]
                                )==0)
                            
    #利润限制共四组
    cc=mip.addConstr(
    (quicksum(c[i]*P[t,i]
            for i in range(num_i)
            for t in range(num_t))
    +quicksum(F[i]*I[i]
             for i in range(num_i))
    -pi)==0    
    )

    #第二组5个
    c2a=mip.addConstr(
        (quicksum(
        -A[t,tt,n,nn,j]*zetaup[nn,j]
        +B[t,tt,n,nn,j]*zetalow[nn,j]
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j)
        for n in range(t+1)
        for nn in range(t+1))
    -R
    -quicksum(d[i][j]*q0[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi
    -quicksum(C[j]*q0[n,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        for n in range(t+1))
    )>=0
    )
    #c2b,c2c,c2d,c2e
    for t in range(num_t):
        for tt in range(num_t):
            for n in range(t+1):
                for nn in range(t+1):
                    for j in range(num_j):
                        if n<t&nn<tt:
                            if nn!=n:
                                mip.addConstr(
                                (-A[t,tt,n,nn,j]
                                 +B[t,tt,n,nn,j]
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                )==0)
                            if nn==n:
                                mip.addConstr(
                                (-A[t,tt,n,nn,j]
                                 +B[t,tt,n,nn,j]
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                 -C[j]
                                )==0)
                        if n==t&nn==tt:
                            if tt!=t:
                                mip.addConstr(
                                (-A[t,tt,n,nn,j]
                                 +B[t,tt,n,nn,j]
                                 +quicksum(q[t,tt,i,j]
                                          for i in range(num_i))
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                )==0)                           
                            if tt==t:
                                mip.addConstr(
                                (-A[t,tt,n,nn,j]
                                 +B[t,tt,n,nn,j]
                                 -eta
                                 +quicksum(q[t,tt,i,j]
                                          for i in range(num_i))
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                 -C[j]
                                )==0)

    #第三组3个
    c3a=mip.addConstr(
    (quicksum(
        -G[t,tt,j]*zetaup[tt,j]+H[t,tt,j]*zetalow[tt,j]
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j))
    -R
    -quicksum(d[i][j]*q0[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi)
    >=0
    )
    
    for t in range(num_t):
        for tt in range(num_t):
            for j in range(num_j):
                if tt!=t:
                    mip.addConstr(
                    (-G[t,tt,j]
                     +H[t,tt,j]
                     +quicksum(d[i][j]*q[t,tt,i,j]
                               for i in range(num_i))
                    )==0)
                if tt==t:
                    mip.addConstr(
                    (-G[t,tt,j]
                     +H[t,tt,j]
                     -eta
                     +quicksum(d[i][j]*q[t,tt,i,j]
                               for i in range(num_i))
                    )==0)

    #第四组5个
    c4a=mip.addConstr(
    (quicksum(
        -o[t,tt,n,nn,j]*zetaup[nn,j]
        +s[t,tt,n,nn,j]*zetalow[nn,j]
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j)
        for n in range(t+1)
        for nn in range(t+1))
    -R
    +quicksum((eta-d[i][j])*q0[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi
    -quicksum(C[j]*q0[n,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        for n in range(t+1))
    )>=0
    )
    #c2b,c2c,c2d,c2e
    for t in range(num_t):
        for tt in range(num_t):
            for n in range(t+1):
                for nn in range(t+1):
                    for j in range(num_j):
                        if n<t&nn<tt:
                            if nn!=n:
                                mip.addConstr(
                                (-o[t,tt,n,nn,j]
                                 +s[t,tt,n,nn,j]
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                )==0)
                            if nn==n:
                                mip.addConstr(
                                (-o[t,tt,n,nn,j]
                                 +s[t,tt,n,nn,j]
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                 -C[j]
                                )==0)
                        if n==t&nn==tt:
                            if tt!=t:
                                mip.addConstr(
                                (-o[t,tt,n,nn,j]
                                 +s[t,tt,n,nn,j]
                                 -quicksum((eta-d[i][j])*q[t,tt,i,j]
                                          for i in range(num_i))
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                )==0)                           
                            if tt==t:
                                mip.addConstr(
                                (-o[t,tt,n,nn,j]
                                 +s[t,tt,n,nn,j]
                                 -quicksum((eta-d[i][j])*q[t,tt,i,j]
                                          for i in range(num_i))
                                 +quicksum(C[j]*q[n,nn,i,j]
                                          for i in range(num_i))
                                 -C[j]
                                )==0)

    #第五组 2个
    c5a=mip.addConstr(
    (quicksum(
        -r[tt,j]*zetaup[tt,j]+w[tt,j]*zetalow[tt,j]
        for tt in range(num_t)
        for j in range(num_j))
    -R
    +quicksum((eta-d[i][j])*q0[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
    -pi)
    >=0
    )

    c5b=mip.addConstrs(
    (-r[tt,j]
     +w[tt,j]
     -quicksum((eta-d[i][j])*q[t,tt,i,j]
        for t in range(num_t)
        for i in range(num_i))
    )==0
    for j in range(num_j)
    for tt in range(num_t)
    )
    #extra 1
    for t in range(num_t):
        for tt in range(num_t):
            for i in range(num_i):
                for j in range(num_j):
                    if tt>=t:
                        mip.addConstr(
                            q[t,tt,i,j]==0
                        )
    #extra 2
    for t in range(num_t):
        for n in range(t):
            for nn in range(t):
                for i in range(num_i):
                    for j in range(num_j):
                        if nn>=n:
                            mip.addConstr(
                                q[n,nn,i,j]==0
                            )
    #第六组2个
    c6a=mip.addConstrs(
    (quicksum(
        x[t,tt,i,j]*zetaup[tt,j]-z[t,tt,i,j]*zetalow[tt,j]
        for tt in range(num_t)
        for j in range(num_j))
    -P[t,i]
    +quicksum(q0[t,i,j]
        for j in range(num_j))
    )<=0
    for i in range(num_i)
    for t in range(num_t)
    )

    c6b=mip.addConstrs(
    (x[t,tt,i,j]
     -z[t,tt,i,j]
     -q[t,tt,i,j]
    )==0
    for i in range(num_i)
    for j in range(num_j)
    for t in range(num_t)
    for tt in range(num_t)
    )
    #第7个
    c7 = mip.addConstrs( 
    P[t,i]-Z[i]*I[i]<=0
    for i in range(num_i)
    for t in range(num_t)
    )

    #第八组2个
    c8a=mip.addConstrs(
    (quicksum(
        -g[t,tt,i,j]*zetaup[tt,j]+h[t,tt,i,j]*zetalow[tt,j]
        for tt in range(num_t))
    +q0[t,i,j]
    )>=0
    for i in range(num_i)
    for j in range(num_j)
    for t in range(num_t)
    )

    c8b=mip.addConstrs(
    (-g[t,tt,i,j]
     +h[t,tt,i,j]
     -q[t,tt,i,j]
    )==0
    for i in range(num_i)
    for j in range(num_j)
    for t in range(num_t)
    for tt in range(num_t)
    )

    #number 9
    c9=mip.addConstrs(
    alpha[t,j] >= epsilon
    for t in range(num_t)
    for j in range(num_j))

    mip.optimize()

    # Print solution
    #print("Optimal solution of decision variables")
    #for v in mip.getVars():
    #    print (v.varName, v.x)

    obj = mip.getObjective()
    print("Objective value")
    print(obj.getValue())
    return obj

    y=np.array([[[0.00 for j in range(num_j)]for i in range(num_i)]for t in range(num_t)])
    for i in range(num_i):
        for j in range(num_j):
            for t in range(num_t):
                y[t,i,j]=q0[t,i,j].x+sum(zetau[tt,j]*q[t,tt,i,j].x
                                  for tt in range(num_t))
    print(y)
    #########################################################
    fullfill=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fmean=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    fup=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    flow=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            #fullfill[t,j]=sum(y[t,i,j] for i in range(num_i))
            fullfill[t,j]=sum(
            (sum(y[n,i,j] for i in range(num_i))-zetau[n,j])
            for n in range(t)
            )+sum(y[t,i,j] for i in range(num_i))
            fmean[t,j]=round(fullfill[t,j]/zetau[t,j],2)
            fup[t,j]=round(fullfill[t,j]/zetaup[t,j],2)
            flow[t,j]=round(fullfill[t,j]/zetalow[t,j],2)
    print("fullfilled demand")
    print(fullfill)
    print("actual demand")
    print(zetau)
    print("fill rate compared to mean of demand")
    print(fmean)
    print("fill rate compared to upper bound of demand")
    print(fup)
    print("fill rate compared to lower bound of demand")
    print(flow)
    ###################################
    L=np.array([[0 for j in range(num_j)]for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            L[t,j]=max(sum(
            (sum(
                y[n,i,j] for i in range(num_i)
            )-zetau[n,j]) 
            for n in range(t+1)
        ),0)
    print("Inventory level of each store")
    print(L)  
###################################
    profit1=(sum(eta*zetau[t,j] 
            for t in range(num_t)
            for j in range(num_j))
        -sum(d[i,j]*y[t,i,j]
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j))
        -pi.x
        -sum(C[j]*y[n,i,j]
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j)
            for n in range(t+1))
        +sum(C[j]*zetau[n,j]
            for t in range(num_t)
            for j in range(num_j)
            for n in range(t+1))
        )
    profit2=(sum(eta*zetau[t,j] 
            for t in range(num_t)
            for j in range(num_j))
        -sum(d[i,j]*y[t,i,j]
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j))
        -pi.x
        )         
    profit3=(sum((eta-d[i,j])*y[t,i,j]
             for t in range(num_t)
             for i in range(num_i)
             for j in range(num_j))
         -pi.x
         -sum(C[j]*y[n,i,j]
            for t in range(num_t)
            for i in range(num_i)
            for j in range(num_j)
            for n in range(t+1))
         +sum(C[j]*zetau[n,j]
            for t in range(num_t)
            for j in range(num_j)
            for n in range(t+1))
        )         
    profit4=(sum((eta-d[i,j])*y[t,i,j]
             for t in range(num_t)
             for i in range(num_i)
             for j in range(num_j))
         -pi.x
    )
    print("profit1")
    print(profit1)
    print("profit2")
    print(profit2)
    print("profit3")
    print(profit3)
    print("profit4")
    print(profit4)
    #################################
    num1=0
    num2=0
    num3=0
    for t in range(num_t):
        for j in range(num_j):
            if fmean[t,j]<f[t,j]:
                num1+=1
            if fup[t,j]<f[t,j]:
                num2+=1
            if flow[t,j]<f[t,j]:
                num3+=1
    per1=num1/(num_t*num_j)
    per2=num2/(num_t*num_j)
    per3=num3/(num_t*num_j)
    print("violation probability")
    print(round(per1*100,2))
    print(round(per2*100,2)) 
    print(round(per3*100,2))
    ################################
    LL=L.reshape(num_t*num_j,1).tolist()
    print('upper inventory level')
    print(max(LL))
    print('average inventory level')
    print(np.mean(LL))
        ##################################
    vio=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    vioup=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    violow=np.array([[1.000 for j in range(num_j)] for t in range(num_t)])
    for t in range(num_t):
        for j in range(num_j):
            vio[t,j]=max(f[t,j]-fmean[t,j],0.000)
            vioup[t,j]=max(f[t,j]-fup[t,j],0.000)
            violow[t,j]=max(f[t,j]-flow[t,j],0.000)
    #没达到的程度        
    viomean=np.mean(vio)
    viostd=np.std(vio)
    vioVaR95 = norm.ppf(1-0.95, viomean, viostd)
    vioVaR99 = norm.ppf(1-0.99, viomean, viostd)
    vioupmean=np.mean(vioup)
    vioupstd=np.std(vioup)
    vioupVaR95 = norm.ppf(1-0.95, vioupmean, vioupstd)
    vioupVaR99 = norm.ppf(1-0.99, vioupmean, vioupstd)
    violowmean=np.mean(violow)
    violowstd=np.std(violow)
    violowVaR95 = norm.ppf(1-0.95, violowmean, violowstd)
    violowVaR99 = norm.ppf(1-0.99, violowmean, violowstd)

    print('mean/std/VaR95/VaR99 compared to mean')
    print(round(viomean,2))
    print(round(viostd,2))
    print(round(vioVaR95,2))
    print(round(vioVaR99,2))
    print('mean/std/VaR95/VaR99 compared to upper')
    print(round(vioupmean,2))
    print(round(vioupstd,2))
    print(round(vioupVaR95,2))
    print(round(vioupVaR99,2))
    print('mean/std/VaR95/VaR99 compared to lower')
    print(round(violowmean,2))
    print(round(violowstd,2))
    print(round(violowVaR95,2))
    print(round(violowVaR99,2))

    

In [5]:
import numpy as np
from gurobipy import *
from scipy.stats import beta
from scipy.stats import norm
import matplotlib.pyplot as plt
import random
import time

def RFDbender(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit):
    
    
    num_i = M
    num_j = N
    num_t = T
    bar_I = np.array([1 for i in range(num_i)])
    # I_binary,initial solution for benders

    Z = np.array([(400*num_j)/num_i for i in range(num_i)])
    # Z'i_capacity
    
    if demandtype=='constant':
        demand = 200+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_t*num_j, 0.01), 2, 4)
        zeta=demand.reshape(num_t,num_j)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(zeta[t,j],0)
    if demandtype=='seasonal':
        demand=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        zeta=np.array([[0 for j in range(num_j)]for t in range(num_t)])
        for t in range(num_t):
            for j in range(num_j):
                if (t-1)%4==0:
                    demand[t] = 400+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
                else:
                    demand[t] = 100+10*beta.pdf(np.arange(0.01, 0.01+0.01*num_j, 0.01), 2, 4)
        for t in range(num_t):
            for j in range(num_j):
                zeta[t,j]=round(demand[t,j],2)

    zetau=zeta
    #U'tj 需求均值

    zetaup=1.1*zetau
    #zetau'jt 需求上界

    zetalow=0.9*zetau
    #zetal'jt 需求下界
  
    eta = price
    # eta_price

    c =procost
    # ci_productioncost

    C =invcost
    # Cj_inventory cost

    d= transcost
    #dij_transportationcost
    
    F= np.array([20 for i in range(num_i)])
    #Fi_fixed construction cost

    f=desiredfillrate
    #ftj_target of fill rate

    R=profit
    #target profit

    num_k = 3
    a = [0.3909, 0.01298,0]
    b = [0, 0.8701, 1]
    #piecewise linear function,k=3

    epsilon=0.01


    params = dict()
    params["num_i"] = num_i
    params["num_j"] = num_j
    params["num_t"] = num_t
    params["num_k"] = num_k
    params["Z"] = Z
    params["zetau"] = zetau
    params["zetaup"] = zetaup
    params["zetalow"] = zetalow
    params["eta"] = eta
    params["c"] = c
    params["C"] = C
    params["d"] = d
    params["F"] = F
    params["f"] = f
    params["R"] = R
    params["a"] = a
    params["b"] = b
    params["epsilon"] = epsilon
    params["bar_I"] = bar_I
    
    #################################
    def MP(Q, E, iter_i, params):
        num_i=params["num_i"]
        num_j=params["num_j"] 
        num_t=params["num_t"] 
        num_k=params["num_k"]
        Z=params["Z"] 
        zetau=params["zetau"] 
        zetaup=params["zetaup"] 
        zetalow=params["zetalow"]
        eta=params["eta"] 
        c=params["c"]
        C=params["C"]
        d=params["d"] 
        F=params["F"] 
        f=params["f"] 
        R=params["R"]
        a=params["a"] 
        b=params["b"] 
        epsilon=params["epsilon"] 
        bar_I=params["bar_I"]
    
        stop = False
        mp = Model("master_problem")
    
        #decision variables
        I = mp.addVars([i for i in range(num_i)],
                    vtype=GRB.BINARY, name="I")
        beta = mp.addVar(vtype=GRB.CONTINUOUS, name="beta")
    
        #objective function
        mp.setObjective(
            beta,GRB.MINIMIZE)
    
    
        # if sp is unbounded,add feasibility cut
        for item in Q.values():
            mp.addConstr(
            (
            quicksum(item["lamda3"][k,t,n,n,j]*a[k]
                    for k in range(num_k)
                    for t in range(num_t)
                    for n in range(t)
                    for j in range(num_j))
            +quicksum(item["lamda3"][k,t,t,t,j]*a[k]*f[t,j]
                    for k in range(num_k)
                    for t in range(num_t)
                    for j in range(num_j))
            +item["lamda4"]*quicksum(F[i]*I[i]
                            for i in range(num_i))
            +item["theta1"]*R
            +quicksum(item["theta2"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta2"][t,t,t,t,j]*(eta+C[j])
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta3"]*R
            +quicksum(item["theta4"][t,t,j]*eta
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta5"]*R
            +quicksum(item["theta6"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta6"][t,t,t,t,j]*C[j]
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta7"]*R
            -quicksum(item["sigma5"][t,i]*Z[i]*I[i]
                     for t in range(num_t)
                     for i in range(num_i))
            +quicksum(item["sigma8"][t,j]*epsilon
                     for t in range(num_t)
                     for j in range(num_j))
            )<=0)


        # if sp is optimal,add optimality cut
        for item in E.values():
            mp.addConstr(
            (
            quicksum(item["lamda3"][k,t,n,n,j]*a[k]
                    for k in range(num_k)
                    for t in range(num_t)
                    for n in range(t)
                    for j in range(num_j))
            +quicksum(item["lamda3"][k,t,t,t,j]*a[k]*f[t,j]
                    for k in range(num_k)
                    for t in range(num_t)
                    for j in range(num_j))
            +item["lamda4"]*quicksum(F[i]*I[i]
                            for i in range(num_i))
            +item["theta1"]*R
            +quicksum(item["theta2"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta2"][t,t,t,t,j]*(eta+C[j])
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta3"]*R
            +quicksum(item["theta4"][t,t,j]*eta
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta5"]*R
            +quicksum(item["theta6"][t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(item["theta6"][t,t,t,t,j]*C[j]
                     for t in range(num_t)
                     for j in range(num_j))
            +item["theta7"]*R
            -quicksum(item["sigma5"][t,i]*Z[i]*I[i]
                     for t in range(num_t)
                     for i in range(num_i))
            +quicksum(item["sigma8"][t,j]*epsilon
                     for t in range(num_t)
                     for j in range(num_j))
            )<=beta)   
        
        mp.optimize()
    
        bar_I = dict()

        if mp.status == GRB.OPTIMAL:
            for i in range(num_i):
                bar_I[i] = 1 if I[i].x > 0.01 else 0
        else:
            print("There is no feasible solution in primal problem")
            stop = True

        return mp, bar_I, stop
    
    ###########################
    def SP(Q, E, bar_I, iter_i, params):
        num_i=params["num_i"]
        num_j=params["num_j"] 
        num_t=params["num_t"] 
        num_k=params["num_k"]
        Z=params["Z"] 
        zetau=params["zetau"] 
        zetaup=params["zetaup"] 
        zetalow=params["zetalow"]
        eta=params["eta"] 
        c=params["c"]
        C=params["C"]
        d=params["d"] 
        F=params["F"] 
        f=params["f"] 
        R=params["R"]
        a=params["a"] 
        b=params["b"] 
        epsilon=params["epsilon"] 
        bar_I=params["bar_I"]
        stop = False
    
        sp = Model("dualsubproblem")
    
        # get unboundedRay
        sp.Params.InfUnbdInfo = 1
    
        #decision variables
        lamda1 = sp.addVars([t for t in range(num_t)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "lamda1")
        lamda2 = sp.addVars([k for k in range(num_k)],
                        [t for t in range(num_t)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "lamda2")
        lamda3 = sp.addVars([k for k in range(num_k)],
                        [t for t in range(num_t)],
                        [n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "lamda3")
        lamda4 = sp.addVar(lb=-1*GRB.INFINITY,
                       vtype=GRB.CONTINUOUS,name = "lamda4")
       
        theta1 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta1")
        theta2 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta2")
        theta3 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta3")
        theta4 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta4")
        theta5 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta5")
        theta6 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta6")
        theta7 = sp.addVar(vtype=GRB.CONTINUOUS,name = "theta7")
        theta8 = sp.addVars([tt for tt in range(num_t)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "theta8")
    
        sigma1 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma1")
        sigma2 = sp.addVars([n for n in range(num_t)],
                        [nn for nn in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma2")
        sigma3 = sp.addVars([t for t in range(num_t)],
                        [i for i in range(num_i)],
                        vtype=GRB.CONTINUOUS,name = "sigma3")
        sigma4 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma4")
        sigma5 = sp.addVars([t for t in range(num_t)],
                        [i for i in range(num_i)],
                        vtype=GRB.CONTINUOUS,name = "sigma5")
        sigma6 = sp.addVars([t for t in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "sigma6")
        sigma7 = sp.addVars([t for t in range(num_t)],
                        [tt for tt in range(num_t)],
                        [i for i in range(num_i)],
                        [j for j in range(num_j)],
                        lb=-1*GRB.INFINITY,
                        vtype=GRB.CONTINUOUS,name = "sigma7")
        sigma8 = sp.addVars([t for t in range(num_t)],
                        [j for j in range(num_j)],
                        vtype=GRB.CONTINUOUS,name = "sigma8")
    
        #objective function
        sp.setObjective(
            (
            quicksum(lamda3[k,t,n,n,j]*a[k]
                    for k in range(num_k)
                    for t in range(num_t)
                    for n in range(t)
                    for j in range(num_j))
            +quicksum(lamda3[k,t,t,t,j]*a[k]*f[t,j]
                    for k in range(num_k)
                    for t in range(num_t)
                    for j in range(num_j))
            +lamda4*quicksum(F[i]*bar_I[i]
                            for i in range(num_i))
            +theta1*R
            +quicksum(theta2[t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(theta2[t,t,t,t,j]*(eta+C[j])
                     for t in range(num_t)
                     for j in range(num_j))
            +theta3*R
            +quicksum(theta4[t,t,j]*eta
                     for t in range(num_t)
                     for j in range(num_j))
            +theta5*R
            +quicksum(theta6[t,t,n,n,j]*C[j]
                     for t in range(num_t)
                     for n in range(t)
                     for j in range(num_j))
            +quicksum(theta6[t,t,t,t,j]*C[j]
                     for t in range(num_t)
                     for j in range(num_j))
            +theta7*R
            -quicksum(sigma5[t,i]*Z[i]*bar_I[i]
                     for t in range(num_t)
                     for i in range(num_i))
            +quicksum(sigma8[t,j]*epsilon
                     for t in range(num_t)
                     for j in range(num_j))
        ),GRB.MAXIMIZE)
    
        #constraints for dual value
        #S
        d1=sp.addConstrs(
        (lamda1[t,j]
         -quicksum(lamda2[k,t,j]
                   for k in range(num_k))
        )==0
        for j in range(num_j)
        for t in range(num_t)
        )    
        d2=sp.addConstrs(
        (lamda1[t,j]*zetau[t,j]
         -quicksum(lamda3[k,t,t,t,j]
                   for k in range(num_k))
        )==0
        for j in range(num_j)
        for t in range(num_t)
        )
        #u,v
        d3=sp.addConstrs(
        (-lamda2[k,t,j]*zetaup[nn,j]
         +lamda3[k,t,n,nn,j]
        )<=0
        for k in range(num_k)
        for t in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
        )
        d4=sp.addConstrs(
        (lamda2[k,t,j]*zetalow[nn,j]
         -lamda3[k,t,n,nn,j]
        )<=0
        for k in range(num_k)
        for t in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
        )
        #A,B
        d5=sp.addConstrs(
        (
        -theta1*zetaup[nn,j]
        -theta2[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
        )
    
        d6=sp.addConstrs(
        (
        theta1*zetalow[nn,j]
        +theta2[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
        )
        #G,H
        d7=sp.addConstrs(
        (
        -theta3*zetaup[tt,j]
        -theta4[t,tt,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j)
        )
        d8=sp.addConstrs(
        (
        theta3*zetalow[tt,j]
        +theta4[t,tt,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for j in range(num_j)
        )
        #o,s
        d9=sp.addConstrs(
        (
        -theta5*zetaup[nn,j]
        -theta6[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
        )    
        d10=sp.addConstrs(
        (
        theta5*zetalow[nn,j]
        +theta6[t,tt,n,nn,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for n in range(t+1)
        for nn in range(t+1)
        for j in range(num_j)
        )
        #r,w
        d11=sp.addConstrs(
        (
        -theta7*zetaup[tt,j]
        -theta8[tt,j]
        )<=0
        for tt in range(num_t)
        for j in range(num_j)
        )    
        d12=sp.addConstrs(
        (
        theta7*zetalow[tt,j]
        +theta8[tt,j]
        )<=0
        for tt in range(num_t)
        for j in range(num_j)
        )
        #x,z
        d13=sp.addConstrs(
        (
        -sigma3[t,i]*zetaup[tt,j]
        +sigma4[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        )    
        d14=sp.addConstrs(
        (
        sigma3[t,i]*zetalow[tt,j]
        -sigma4[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        )
        #g,h
        d15=sp.addConstrs(
        (
        -sigma6[t,i,j]*zetaup[tt,j]
        -sigma7[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        )    
        d16=sp.addConstrs(
        (
        sigma6[t,i,j]*zetalow[tt,j]
        +sigma7[t,tt,i,j]
        )<=0
        for t in range(num_t)
        for tt in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        )
        #qt
        d17=sp.addConstrs(
            (
            quicksum(lamda2[k,t,j]*a[k]
                     for k in range(num_k))
            -theta1*d[i,j]
            -theta1*C[j]
            -theta3*d[i,j]
            +theta5*(eta-d[i,j])
            -theta5*C[j]
            +theta7*(eta-d[i,j])
            -sigma3[t,i]
            +sigma6[t,i,j]
            )==0
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j)
        ) 
        #qtt
        for t in range(num_t):
            for tt in range(num_t):
                for i in range(num_i):
                    for j in range(num_j):
                        if tt>=t:
                            sp.addConstr(
                            (
                            quicksum(lamda3[k,t,t,tt,j]*a[k]
                                    for k in range(num_k))
                                +theta2[t,tt,t,tt,j]*C[j]
                                +theta2[t,tt,t,tt,j]
                                +theta4[t,tt,j]*d[i,j]
                                +theta6[t,tt,t,tt,j]*C[j]
                                -theta6[t,tt,t,tt,j]*(eta-d[i,j])
                                -theta8[tt,j]*(eta-d[i,j])
                                +sigma1[t,tt,i,j]
                                -sigma4[t,tt,i,j]
                                -sigma7[t,tt,i,j]
                                )==0
                        )
                        if tt<t:
                            sp.addConstr(
                            (
                            quicksum(lamda3[k,t,t,tt,j]*a[k]
                                    for k in range(num_k))
                                +theta2[t,tt,t,tt,j]*C[j]
                                +theta2[t,tt,t,tt,j]
                                +theta4[t,tt,j]*d[i,j]
                                +theta6[t,tt,t,tt,j]*C[j]
                                -theta6[t,tt,t,tt,j]*(eta-d[i,j])
                                -theta8[tt,j]*(eta-d[i,j])
                                -sigma4[t,tt,i,j]
                                -sigma7[t,tt,i,j]
                                )==0
                            )
        #qn
        d19=sp.addConstrs(
            (
            quicksum(lamda2[k,t,j]*a[k]
                    for k in range(num_k))
            -theta1*C[j]
            -theta5*C[j]
            )==0
        for t in range(num_t)
        for n in range(t)
        for i in range(num_i)
        for j in range(num_j)
        )
        #qnn
        for t in range(num_t):
            for tt in range(num_t):
                for n in range(t):
                    for nn in range(t):
                        for i in range(num_i):
                            for j in range(num_j):
                                if nn>=n:
                                    sp.addConstr(
                                    (
                                    quicksum(lamda3[k,t,n,nn,j]*a[k]
                                            for k in range(num_k))
                                    +theta2[t,tt,n,nn,j]*C[j]
                                    +theta6[t,tt,n,nn,j]*C[j]
                                    +sigma2[n,nn,i,j]
                                    )==0
                                )
                                if nn<n:
                                    sp.addConstr(
                                    (
                                    quicksum(lamda3[k,t,n,nn,j]*a[k]
                                            for k in range(num_k))
                                    +theta2[t,tt,n,nn,j]*C[j]
                                    +theta6[t,tt,n,nn,j]*C[j]
                                    )==0
                                )
    
        #P
        d21=sp.addConstrs(
            (
            -lamda4*c[i]
            +sigma3[t,i]
            -sigma5[t,i]
            )==0
        for t in range(num_t)
        for i in range(num_i)
        ) 
        #pi
        d22=sp.addConstr(
            (
            lamda4-theta1-theta3-theta5-theta7
            )==0
        )
        #alpha
        d23=sp.addConstrs(
            (
            quicksum(lamda3[k,t,t,t,j]*b[k]
                    for k in range(num_k))
            +sigma8[t,j]
            )==1
        for t in range(num_t)
        for j in range(num_j)
        )
        
        sp.optimize()
    
        #get dual value
        #    constrs_name=sp.getAttr("ConstrName", sp.getConstrs())
        #    y=sp.getAttr("Pi", sp.getConstrs())
        
        #print("sub problem's status:"+str(sp.status)
        if sp.status == GRB.UNBOUNDED:
            item = dict()
            item["lamda1"] = {(t,j):lamda1[t,j].unbdRay
                          for t in range(num_t)
                          for j in range(num_j)}
            item["lamda2"] = {(k,t,j):lamda2[k,t,j].unbdRay
                          for k in range(num_k)
                          for t in range(num_t)
                          for j in range(num_j)}
            item["lamda3"] = {(k,t,n,nn,j):lamda3[k,t,n,nn,j].unbdRay
                          for k in range(num_k)
                          for t in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
            item["lamda4"] = lamda4.unbdRay
        
            item["theta1"] = theta1.unbdRay
            item["theta2"] = {(t,tt,n,nn,j):theta2[t,tt,n,nn,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
            item["theta3"] = theta3.unbdRay
            item["theta4"] = {(t,tt,j):theta4[t,tt,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for j in range(num_j)}
            item["theta5"] = theta5.unbdRay
            item["theta6"] = {(t,tt,n,nn,j):theta6[t,tt,n,nn,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
            item["theta7"] = theta7.unbdRay
            item["theta8"] = {(tt,j):theta8[tt,j].unbdRay 
                          for tt in range(num_t)
                          for j in range(num_j)}
        
            item["sigma1"] = {(t,tt,i,j):sigma1[t,tt,i,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma2"] = {(n,nn,i,j):sigma2[n,nn,i,j].unbdRay 
                          for n in range(num_t)
                          for nn in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma3"] = {(t,i):sigma3[t,i].unbdRay 
                          for t in range(num_t)
                          for i in range(num_i)}
            item["sigma4"] = {(t,tt,i,j):sigma4[t,tt,i,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma5"] = {(t,i):sigma5[t,i].unbdRay 
                          for t in range(num_t)
                          for i in range(num_i)}
            item["sigma6"] = {(t,i,j):sigma6[t,i,j].unbdRay 
                          for t in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma7"] = {(t,tt,i,j):sigma7[t,tt,i,j].unbdRay 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma8"] = {(t,j):sigma8[t,j].unbdRay 
                          for t in range(num_t)
                          for j in range(num_j)}        
        
            Q[iter_i] = item
            print("Add feasible cut")

        elif sp.status == GRB.OPTIMAL:
            item = dict()
            item["lamda1"] = {(t,j):lamda1[t,j].x
                          for t in range(num_t)
                          for j in range(num_j)}
            item["lamda2"] = {(k,t,j):lamda2[k,t,j].x
                          for k in range(num_k)
                          for t in range(num_t)
                          for j in range(num_j)}
            item["lamda3"] = {(k,t,n,nn,j):lamda3[k,t,n,nn,j].x
                          for k in range(num_k)
                          for t in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
            item["lamda4"] = lamda4.x
                
            item["theta1"] = theta1.x
            item["theta2"] = {(t,tt,n,nn,j):theta2[t,tt,n,nn,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
            item["theta3"] = theta3.x
            item["theta4"] = {(t,tt,j):theta4[t,tt,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for j in range(num_j)}
            item["theta5"] = theta5.x
            item["theta6"] = {(t,tt,n,nn,j):theta6[t,tt,n,nn,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for n in range(num_t)
                          for nn in range(num_t)
                          for j in range(num_j)}
            item["theta7"] = theta7.x
            item["theta8"] = {(tt,j):theta8[tt,j].x
                          for tt in range(num_t)
                          for j in range(num_j)}
        
            item["sigma1"] = {(t,tt,i,j):sigma1[t,tt,i,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma2"] = {(n,nn,i,j):sigma2[n,nn,i,j].x 
                          for n in range(num_t)
                          for nn in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma3"] = {(t,i):sigma3[t,i].x
                          for t in range(num_t)
                          for i in range(num_i)}
            item["sigma4"] = {(t,tt,i,j):sigma4[t,tt,i,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma5"] = {(t,i):sigma5[t,i].x
                          for t in range(num_t)
                          for i in range(num_i)}
            item["sigma6"] = {(t,i,j):sigma6[t,i,j].x 
                          for t in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma7"] = {(t,tt,i,j):sigma7[t,tt,i,j].x 
                          for t in range(num_t)
                          for tt in range(num_t)
                          for i in range(num_i)
                          for j in range(num_j)}
            item["sigma8"] = {(t,j):sigma8[t,j].x 
                          for t in range(num_t)
                          for j in range(num_j)}
        
            E[iter_i] = item
            print("Add optimal cut")

        else:
            print("Wrong sub-problem status  " + str(sp.status))
            stop = True
        
    #   return sp,constrs_name,y,stop
        return sp,stop
    
    start =time.clock()
    #中间写上代码块
    E_set = dict()
    Q_set = dict()

    LB = -1 * 1e10
    UB = 1e10

    for iter_i in range(1000):
        if np.abs(UB - LB) < 0.01:
            print("Optimal")
            break
        
        print("iteration at " + str(iter_i))
    
        sp,stop = SP(Q_set, E_set, bar_I, iter_i, params)        
        print("Q  " + str(len(Q_set.keys())))
        print("E  " + str(len(E_set.keys())))
        if stop:
            print("Wrong slave problem")
            break
        #item = E_set.get(iter_i, False)
        sp_optimal = sp.objVal 
        print("slave dual objective value")
        print(sp_optimal)
        UB = min(UB, sp_optimal)
        # get dual value of contraints
        #print(constrs_name)
        #print(y)
        #主问题输出
        mp, bar_I, stop = MP(Q_set, E_set, iter_i, params)
        print("bar_I")
        print(bar_I)
        print("master objective value")
        print(mp.objVal)
        if stop:
            print("wrong master problem")
            break
        
        LB = mp.objVal
        print("UB " + str(UB))
        print("LB " + str(LB))
    

    

In [9]:
#R(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit)
M = 40
#supply,40 warehouses
N = 1200
#demand,1200 suning stores
T = 52
#period，52 weeks

num_i = M
num_j = N
num_t = T

demandtype='seasonal'

price = 50
# eta_price

procost =np.array([30 for i in range(num_i)])
# ci_productioncost

invcost =np.array([10 for j in range(num_j)])
# Cj_inventory cost

transcost= np.array([[
    random.randint(3,8) 
    for j in range(num_j)] 
    for i in range(num_i)])
#transcost= np.array([
#[6,5,6,8,3,7,3,8,8,4,7,6,3,8,6,3],
#[3,8,6,5,6,6,4,4,4,4,8,8,6,3,3,7],
#[3,7,8,3,7,6,5,3,4,7,6,8,3,3,5,8],
#[7,7,8,4,4,4,5,8,7,3,6,6,3,6,5,7]])

print(transcost)
#dij_transportationcost
    
desiredfillrate=np.array([[0.8
    for j in range(num_j)]
    for t in range(num_t)])
    #ftj_target of fill rate

profit=0

#target profit


[[5 3 3 ..., 8 4 3]
 [6 7 6 ..., 8 7 5]
 [4 6 7 ..., 4 7 4]
 ..., 
 [5 6 3 ..., 8 6 6]
 [4 7 7 ..., 4 7 3]
 [6 3 8 ..., 7 3 4]]


In [10]:
demandtype='seasonal'

In [11]:
start =time.clock()
#中间写上代码块

RP(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit)
end = time.clock()
    
print('time')
print(end-start)

Optimize a model with 1784960 rows, 15665720 columns and 78071040 nonzeros
Variable types: 15665680 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 15.12 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: -0 

Best objective -0.000000000000e+00, best bound -, gap -


GurobiError: Out of memory

In [9]:
start =time.clock()
#中间写上代码块

RPF(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,profit)
end = time.clock()
    
print('time')
print(end-start)

Optimize a model with 1632 rows, 7092 columns and 11008 nonzeros
Variable types: 7088 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+00]
Presolve removed 1344 rows and 6448 columns
Presolve time: 0.01s
Presolved: 288 rows, 644 columns, 5280 nonzeros
Variable types: 640 continuous, 4 integer (4 binary)

Root relaxation: objective 2.737363e+04, 273 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 27373.6300    0    2          - 27373.6300      -     -    0s
H    0     0                    27370.960000 27373.6300  0.01%     -    0s

Explored 1 nodes (273 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 27371 

Optimal solution found (tolerance 1.00e-04)
Best 

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1907: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1908: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc


In [27]:
start =time.clock()
#中间写上代码块

RF(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,120000)
end = time.clock()
    
print('time')
print(end-start)


Optimize a model with 4229 rows, 12325 columns and 23049 nonzeros
Variable types: 12321 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+05]
Presolve removed 2960 rows and 9568 columns
Presolve time: 0.03s
Presolved: 1269 rows, 2757 columns, 12969 nonzeros
Variable types: 2753 continuous, 4 integer (4 binary)

Root relaxation: objective 9.558845e+00, 2269 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.55885    0    2          -    9.55885      -     -    0s
H    0     0                       9.5594834    9.55885  0.01%     -    0s

Explored 1 nodes (2269 simplex iterations) in 0.12 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 9.55948 

Optimal solution found (tolerance 1.00e

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1907: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1908: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc


In [30]:
start =time.clock()
#中间写上代码块

RFD(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,120000)
end = time.clock()
    
print('time')
print(end-start)

Optimize a model with 35461 rows, 335013 columns and 288521 nonzeros
Variable types: 335009 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+05]
Presolve removed 27346 rows and 317984 columns
Presolve time: 0.24s
Presolved: 8115 rows, 17029 columns, 54407 nonzeros
Variable types: 17025 continuous, 4 integer (4 binary)

Root relaxation: objective 1.280000e+00, 3437 iterations, 0.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       1.2800000    1.28000  0.00%     -    0s

Explored 0 nodes (3437 simplex iterations) in 0.54 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 1.28 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.280000000000e+00, best bound 1.280000000000e+00

fill rate compared to mean of demand
[[ 4.59  4.45  3.07  1.34  4.34  4.4   0.86  4.25  4.23  0.86  0.88  4.25
   4.15  4.23  0.88  0.88]
 [ 0.9   0.88  0.88  0.88  0.88  0.91  0.88  0.88  0.88  0.88  0.88  0.91
   0.89  0.91  0.89  0.89]
 [ 1.38  1.37  1.37  1.36  1.36  1.36  1.35  1.35  1.35  1.35  1.34  1.34
   1.34  1.31  1.31  1.33]
 [ 2.58  1.47  1.47  1.94  1.46  1.46  1.45  1.45  1.45  1.45  1.44  1.44
   1.44  2.54  1.43  1.43]
 [ 1.58  5.23  1.57  1.56  5.1   5.08  2.49  2.53  3.44  1.55  4.93  4.91
   4.89  1.54  4.79  1.53]
 [ 1.08  1.08  1.08  1.08  1.09  1.09  1.09  1.09  1.09  1.09  1.09  1.09
   1.09  1.09  1.07  1.47]
 [ 3.15  3.14  3.13  3.14  3.11  3.11  3.12  3.09  3.09  3.08  3.08  3.07
   3.07  3.07  3.07  3.07]
 [ 2.15  2.14  2.13  2.14  2.11  2.11  2.12  2.09  2.09  2.08  2.08  2.07
   2.07  2.07  2.07  2.07]]
fill rate compared to upper bound of demand
[[ 4.17  4.05  2.8   1.22  3.94  4.    0.78  3.87  3.85  0.78  0.8   3.87
   3.78  3.85  0.8   0.8 ]
 [ 0.82  

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1907: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1908: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc


In [12]:
start =time.clock()
#中间写上代码块

RFDbender(M,N,T,demandtype,price,procost,invcost,transcost,desiredfillrate,45915.3)
end = time.clock()
    
print('time')
print(end-start)

iteration at 0
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 221217 rows, 174405 columns and 663013 nonzeros
Coefficient statistics:
  Matrix range     [1e-02, 5e+02]
  Objective range  [1e-02, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 149376 rows and 146256 columns
Presolve time: 1.33s
Presolved: 71841 rows, 28149 columns, 320389 nonzeros

Ordering time: 0.02s

Barrier statistics:
 Dense cols : 92
 Free vars  : 144
 AA' NZ     : 3.350e+05
 Factor NZ  : 7.887e+05 (roughly 16 MBytes of memory)
 Factor Ops : 7.336e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.08813210e+04 -9.48258440e+00  2.33e-01 3.84e+02  2.23e+02     2s
   1  -3.44370148e+04  8.32288547e+03